# PROSPERDATA

This notebook creates a dynamic foreground database and links it to ecoinvent and biosphere data

environment: premise2

# project

In [1]:
import bw2data as bd 
import bw2io as bi
import bw2calc as bc

In [2]:
bd.projects.set_current('ecoinvent_391_6')

In [ ]:
list(bd.databases)

In [4]:
#import bw_temporalis as bwt wenn invgen
import numpy as np

In [5]:
# This class is copied from bw_temporalis 

import numpy.typing as npt

class TemporalDistribution:

    def __init__(
              self, date: npt.NDArray[np.datetime64 | np.timedelta64], amount: npt.NDArray
        ):
            datetime_type = np.dtype("datetime64[s]")
            timedelta_type = np.dtype("timedelta64[s]")
            if not isinstance(date, np.ndarray) or not isinstance(amount, np.ndarray):
                raise ValueError("Invalid input types")
            elif not date.shape == amount.shape:
                raise ValueError("Shapes of input `date` and `amount` not identical")
            elif not (
                np.issubdtype(date.dtype, np.datetime64)
                or np.issubdtype(date.dtype, np.timedelta64)
            ):
                raise ValueError(f"Incorrect `date` dtype ({date.dtype})")
            elif not len(date):
                raise ValueError("Empty array")

            # Conversion needed for `temporal_convolution` functions
            self.amount = amount.astype(np.float64)

            if np.issubdtype(date.dtype, np.datetime64):
               self.date = date.astype(datetime_type)
               self.base_time_type = datetime_type
            elif np.issubdtype(date.dtype, np.timedelta64):
               self.date = date.astype(timedelta_type)
               self.base_time_type = timedelta_type
            else:
               raise ValueError("`date` must be numpy datetime or timedelta array")
    
    @property
    def total(self) -> float:
        return float(self.amount.sum())

# ecoinvent

In [6]:
ei = bd.Database("ecoinvent_391_6")
len(ei)

21238

In [7]:
diesel = [act for act in ei if act['name']=='market for diesel' and 'Europe without Switzerland' in act['location']][0]
diesel.as_dict()

{'comment': "This is a market activity. Each market represents the consumption mix of a product in a given geography, connecting suppliers with consumers of the same product in the same geographical area. Markets group the producers and also the imports of the product (if relevant) within the same geographical area. They also account for transport to the consumer and for the losses during that process, when relevant.\nThis is the market for  'diesel', in the geography of Europe without Switzerland.\nInventory for the distribution of petroleum product to the final consumer (household, car, power plant, etc.) including all necessary transports.\n[This dataset was already contained in the ecoinvent database version 2. It was not individually updated during the transfer to ecoinvent version 3. Life Cycle Impact Assessment results may still have changed, as they are affected by changes in the supply chain, i.e. in other datasets. This dataset was generated following the ecoinvent quality gu

In [8]:
pbus = [act for act in ei if act['name']=='bus production' and 'RER' in act['location']][0]
pbus.as_dict()

{'comment': 'The dataset represents the production of one bus (material composition, energy consumption and emissions of the manufacturing). The disposal of bus is included too.\nGeography:  Vehicle manufacturing data are taken from one production site in  Germany.\nTechnology:  Current production technology, environmental issues have been addressed in the past.\nTime period:  The material composition of the modelled bus in this research is based on the material composition of a Volvo bus, (available from product declaration Volvo Truck Corporation (2004)). Energy expenditures for stationary combustion processes are available from environmental reports of a\nGerman lorry manufacturer MAN Nutzfahrzeuge AG (2000) and MAN Nutzfahrzeuge AG (2000).\nReferences: \nVolvo Truck Corporation (2004) Environmental product declaration: Volvo 8500 Low-Entry. Download of March from http://www.volvo.com/bus/global/engb/\nvolvogroup/core+values/environment/downloads.htm.\nMAN Nutzfahrzeuge AG (2000) Um

In [9]:
batte = [act for act in ei if act['name']=='battery production, Li-ion, LiMn2O4, rechargeable, prismatic' and 'GLO' in act['location']][0]
batte.as_dict()

{'comment': 'The dataset can be used for a lithium-ion battery, e.g. for mechanical drive of an electrical vehicle. The modelled battery has a mass of 280 kg, contains 216 cells and holds a capacity of 32 kWh and thus an energy density of 0.114 kWh/kg. Data are based on measurements and scaled to 1kg of battery. For infrastructure, the ecoinvent dataset "electronic component production plant" is accounted.\nNone\nFor more information refer to:\n- Notter D. A., Gauch M., Widmer R., Wäger P., Stamp A., Zah R. and Althaus H.-J. (2010) Contribution of Li-ion batteries to the environmental impact of electric vehicles. In: Environmental Science & Technology.\n- Technical Specifications: https://www.protoscar.com/pdf/LAMPO2_Technical_Specifications.pdf\nGeography:  Some European datasets are used for global processes.\nTechnology:  This dataset includes a method of producing a lithium ion battery. During the charge process, lithium ion move from the cathode (LiMn2O4) into the electrolyte (LiP

In [10]:
carbfib = [act for act in ei if act['name']=='carbon fibre reinforced plastic, injection moulded' and 'GLO' in act['location']][0]
carbfib.as_dict()

{'comment': 'The dataset represents the production of one kilogram of carbon fibre reinforced plastic for use in aircraft manufacture. The data is based on information presented as part of the supplementary information of Cox et al. (2018).\nCarbon fibre reinforced plastic (CFRP) is a structural material formed from extremely thin carbon fibres in a polymeric matrix. The composite has high tensile strength, low thermal expansion and a very high strength-to-weight ratio. These properties have made it an important structural material in the aircraft manufacturing industry (Uhlmann et al., 2014; Pramanik et al., 2017).\nThe material and energy required for the production of carbon fibres is considered, along with the waste and environmental emissions released by this process. Transformation of these fibres into carbon fibre reinforced plastics, as used in aircraft manufacture are then considered (Cox et al., 2018).\nThis activity was created as an input for the aircraft production activit

bus fleet production

# biosphere

In [11]:
bs = bd.Database('biosphere3')

In [12]:
co2 = [act for act in bs if act['name']=='Carbon dioxide, fossil' and act['categories']==('air',)][0]
co2

'Carbon dioxide, fossil' (kilogram, None, ('air',))

In [13]:
kwind = [act for act in bs if act['name']=='Energy, kinetic (in wind), converted'][0]
kwind

'Energy, kinetic (in wind), converted' (megajoule, None, ('natural resource', 'in air'))

In [14]:
khydr = [act for act in bs if act['name']=='Energy, potential (in hydropower reservoir), converted'][0]
khydr

'Energy, potential (in hydropower reservoir), converted' (megajoule, None, ('natural resource', 'in water'))

In [15]:
kphot = [act for act in bs if act['name']=='Energy, solar, converted'][0]
kphot


'Energy, solar, converted' (megajoule, None, ('natural resource', 'in air'))

# parametres

In [100]:
#-------------------------------------
# time span of the calculation
#-------------------------------------
initio = np.datetime64('2024-01-01') #
finis = np.datetime64('2055-01-01')  #
passus = 1            
#-------------------------------------
tempo = np.arange(initio, finis, np.timedelta64(passus, 'Y'), dtype='datetime64[Y]')

# constant demand of functional unit 1000 person kilometer per year
vfu = []
for t in tempo:
    v = 1000 # pkm/a
    vfu.append(v) 
    
tdfu = TemporalDistribution(np.array(tempo, dtype = 'datetime64[Y]'), np.array(vfu))
#----------------------------------------------------------------------------------------------------------------------------------------------
    
#----------------------------------------------------------------------------------------------------------------------------------------------
#===================================== Application busses ==========================================
#------------------------------------------------------------------------------------------------
tdBusse = TemporalDistribution(np.array(tempo, dtype = 'datetime64[Y]'), np.array(vfu))
sumBusse = tdBusse.total

# Begin of bus purchase:
startTB = np.datetime64('2026-01-01')
startBED = np.datetime64('2030-01-01') 
startHFC = np.datetime64('2034-01-01') 
startBEO = np.datetime64('2042-01-01')

# Discontinue of combustion engines:
stillICE = np.datetime64('2040-01-01') #
# Life span of busses
ld = 12 * np.timedelta64(365, 'D')

ldICE = ld
ldBED = ld
ldBEO = ld
ldTB = ld
ldHFC = ld

dies = (stillICE - initio)
anni = dies / np.timedelta64(365, 'D') #division by 365.2425 not possible!
dauerICE = anni.astype(int) # rounded value

#---------------------------------------
# Fleet size
gflt = 0.0001875 # at 1 pMm 

# yearly purchase
flt = 0.000015625 # at 1 pMm 
volc = 1 # no combustion engines since startBE
vold = 1
volo = 1
volt = 1
#---------------------------------------
# Capacity over the whole life span 5 333 333 pkm/Bus
llei = 1


vICE = []
pICE = []
eICE = []
vact = flt * (ldICE / np.timedelta64(365, 'D'))
for t in tempo:
    if t < startTB:
        nv = flt
        vl = flt
    elif t < startTB + ldICE:
        nv = 0
        vl = flt
    else: 
        nv = 0
        vl = 0
    vact = vact + nv - vl
    vICE.append(vact/gflt)
    pICE.append(nv/llei)
    eICE.append(vl/llei)

tdICE = TemporalDistribution(np.array(tempo, dtype = 'datetime64[Y]'), np.array(vICE))
tpICE = TemporalDistribution(np.array(tempo, dtype = 'datetime64[Y]'), np.array(pICE))
teICE = TemporalDistribution(np.array(tempo, dtype = 'datetime64[Y]'), np.array(eICE))
sumICE = tdICE.total
supICE = tpICE.total
sueICE = teICE.total

vBED = []
pBED = []
eBED = []
vact = 0
for t in tempo:
    if t < startBED:
        nv = 0
        vl = 0
    elif t < startHFC:
        nv = flt
        vl = 0
    elif t < startBED + ldBED:
        nv = 0
        vl = 0
    elif t < startHFC + ldBED:
        nv = 0
        vl = flt
    else:
        nv = 0
        vl = 0
    vact = vact + nv - vl
    vBED.append(vact/gflt)
    pBED.append(nv/llei)
    eBED.append(vl/llei)

tdBED = TemporalDistribution(np.array(tempo, dtype = 'datetime64[Y]'), np.array(vBED))
tpBED = TemporalDistribution(np.array(tempo, dtype = 'datetime64[Y]'), np.array(pBED))
teBED = TemporalDistribution(np.array(tempo, dtype = 'datetime64[Y]'), np.array(eBED))
sumBED = tdBED.total
supBED = tpBED.total
sueBED = teBED.total

vHFC = []
pHFC = []
eHFC = []
vact = 0
for t in tempo:
    if t < startHFC:
        nv = 0
        vl = 0
    elif t < stillICE:
        nv = flt
        vl = 0
    elif t < startHFC + ldHFC:
        nv = 0
        vl = 0
    elif t < stillICE + ldHFC:
        nv = 0
        vl = flt
    else:
        nv = 0
        vl = 0
    vact = vact + nv - vl
    vHFC.append(vact/gflt)
    pHFC.append(nv/llei)
    eHFC.append(vl/llei)

tdHFC = TemporalDistribution(np.array(tempo, dtype = 'datetime64[Y]'), np.array(vHFC))
tpHFC = TemporalDistribution(np.array(tempo, dtype = 'datetime64[Y]'), np.array(pHFC))
teHFC = TemporalDistribution(np.array(tempo, dtype = 'datetime64[Y]'), np.array(eHFC))
sumHFC = tdHFC.total
supHFC = tpHFC.total
sueHFC = teHFC.total

vBEO = []
pBEO = []
eBEO = []
vact = 0
for t in tempo:
    if t < startBEO:
        nv = 0
        vl = 0
    elif t < startBED + ldBED:
        nv = flt
        vl = 0
    elif t < startHFC + ldHFC:
        nv = 0
        vl = 0
    elif t < startBEO + ldBEO:
        nv = 0
        vl = flt
    else:
        nv = flt
        vl = flt
    vact = vact + nv - vl
    vBEO.append(vact/gflt)
    pBEO.append(nv/llei)
    eBEO.append(vl/llei)

tdBEO = TemporalDistribution(np.array(tempo, dtype = 'datetime64[Y]'), np.array(vBEO))
tpBEO = TemporalDistribution(np.array(tempo, dtype = 'datetime64[Y]'), np.array(pBEO))
teBEO = TemporalDistribution(np.array(tempo, dtype = 'datetime64[Y]'), np.array(eBEO))
sumBEO = tdBEO.total
supBEO = tpBEO.total
sueBEO = teBEO.total

vTB = []
pTB = []
eTB = []
vact = 0
for t in tempo:
    if t < startTB:
        nv = 0
        vl = 0
    elif t < startBED:
        nv = flt
        vl = 0
    elif t < startTB + ldTB:
        nv = 0
        vl = 0
    else:
        nv = flt
        vl = flt
    vact = vact + nv - vl
    vTB.append(vact/gflt)
    pTB.append(nv/llei)
    eTB.append(vl/llei)

tdTB = TemporalDistribution(np.array(tempo, dtype = 'datetime64[Y]'), np.array(vTB))
tpTB = TemporalDistribution(np.array(tempo, dtype = 'datetime64[Y]'), np.array(pTB))
teTB = TemporalDistribution(np.array(tempo, dtype = 'datetime64[Y]'), np.array(eTB))
sumTB = tdTB.total
supTB = tpTB.total
sueTB = teTB.total


In [101]:
# ----------------------------------------------------------------------------------------------
# Contribution of power plants
fwind = []
for t in tempo:
    fwind.append(1/3) 
fhydr = []
for t in tempo:
    fhydr.append(1/3) 
fphot = []
for t in tempo:
    fphot.append(1/3) 
# ----------------------------------------------------------------------------------------------
# power plants per kWh
pwind = 3539071 # kWh/a
phydr = 15484000000 # kWh/a
pphot = 2766 #kWh/a
tdwind = TemporalDistribution(np.array(tempo, dtype = 'datetime64[Y]'), np.array(fwind))
tdhydr = TemporalDistribution(np.array(tempo, dtype = 'datetime64[Y]'), np.array(fhydr))
tdphot = TemporalDistribution(np.array(tempo, dtype = 'datetime64[Y]'), np.array(fphot))
tnwind = TemporalDistribution(np.array(tempo, dtype = 'datetime64[Y]'), np.array([f/pwind for f in fwind]))
tnhydr = TemporalDistribution(np.array(tempo, dtype = 'datetime64[Y]'), np.array([f/phydr for f in fhydr]))
tnphot = TemporalDistribution(np.array(tempo, dtype = 'datetime64[Y]'), np.array([f/pphot for f in fphot]))
sunwind = tnwind.total
sunhydr = tnhydr.total
sunphot = tnphot.total
print(tnwind.amount)
print(tnhydr.amount)
print(tnphot.amount)

[9.41866759e-08 9.41866759e-08 9.41866759e-08 9.41866759e-08
 9.41866759e-08 9.41866759e-08 9.41866759e-08 9.41866759e-08
 9.41866759e-08 9.41866759e-08 9.41866759e-08 9.41866759e-08
 9.41866759e-08 9.41866759e-08 9.41866759e-08 9.41866759e-08
 9.41866759e-08 9.41866759e-08 9.41866759e-08 9.41866759e-08
 9.41866759e-08 9.41866759e-08 9.41866759e-08 9.41866759e-08
 9.41866759e-08 9.41866759e-08 9.41866759e-08 9.41866759e-08
 9.41866759e-08 9.41866759e-08 9.41866759e-08]
[2.15275984e-11 2.15275984e-11 2.15275984e-11 2.15275984e-11
 2.15275984e-11 2.15275984e-11 2.15275984e-11 2.15275984e-11
 2.15275984e-11 2.15275984e-11 2.15275984e-11 2.15275984e-11
 2.15275984e-11 2.15275984e-11 2.15275984e-11 2.15275984e-11
 2.15275984e-11 2.15275984e-11 2.15275984e-11 2.15275984e-11
 2.15275984e-11 2.15275984e-11 2.15275984e-11 2.15275984e-11
 2.15275984e-11 2.15275984e-11 2.15275984e-11 2.15275984e-11
 2.15275984e-11 2.15275984e-11 2.15275984e-11]
[0.00012051 0.00012051 0.00012051 0.00012051 0.00012

bus transport

In [102]:
banca = 'bustransB'

In [103]:
bd.Database(banca).write(
    {
        (banca, 'passenger transport'): 
        {
            'code': 'passenger transport', 
            'name': 'passenger transport', 
            'kind': 'other',
            'exchanges': 
            [
                {
                    'amount': sumBusse, # pkm/a
                    'input': (banca, 'bus fleet'), 
                    'temporal distribution': tdBusse,
                    'type': 'technosphere'
                    
                }
            ]
        }
        ,
        (banca, 'bus fleet'): 
        {
            'code': 'bus fleet', 
            'name': 'bus fleet', 
            'kind': 'other',
            'exchanges': 
            [
                {
                    'amount': sumICE, # Bus/fleet
                    'input': (banca, 'bus, internal combustion engine'), 
                    'temporal distribution': tdICE,
                    'type': 'technosphere'
                }
                , 
                {
                    'amount': sumBED, 
                    'input': (banca, 'bus, battery electric, depot charging'), 
                    'temporal distribution': tdBED,
                    'type': 'technosphere'
                }
                , 
                {
                    'amount': sumBEO, 
                    'input': (banca, 'bus, battery electric, opportunity charging'), 
                    'temporal distribution': tdBEO,
                    'type': 'technosphere'
                }
                ,
                {
                    'amount': sumTB, 
                    'input': (banca, 'bus, trolley bus'), 
                    'temporal distribution': tdTB,
                    'type': 'technosphere'
                }
                ,
                {
                    'amount': sumHFC, 
                    'input': (banca, 'bus, hydrogen fuel cell'), 
                    'temporal distribution': tdHFC,
                    'type': 'technosphere'
                }
            ]
        }
        , 
        (banca, 'bus, internal combustion engine'): 
        {
            'code': 'bus, internal combustion engine', 
            'name': 'bus, internal combustion engine', 
            'kind': 'other',
            'exchanges': 
            [
                {
                    'amount': 0.0036, # kg/pkm
                    'input': diesel,
                    'temporal distribution': [(0, 0.0036)],
                    'type': 'technosphere'
                }
                , 
                {
                    'amount': 0.01077, # kg/pkm
                    'input': ('biosphere3', '349b29d1-3e58-4c66-98b9-9d1a076efd2e'), 
                    'temporal distribution': [(0, 0.01077)],
                    'type': 'biosphere'
                }
            ]
        }
        , 
        (banca, 'bus, battery electric, depot charging'): 
        {
            'code': 'bus, battery electric, depot charging', 
            'name': 'bus, battery electric, depot charging', 
            'kind': 'other',
            'exchanges': 
            [
                {
                    'amount': 0.0158, # kWh/pkm
                    'input': (banca, 'electricity mix'), 
                    'temporal distribution': [(0, 0.0158)],
                    'type': 'technosphere'
                }
            ]
        }
        , 
        (banca, 'bus, battery electric, opportunity charging'): 
        {
            'code': 'bus, battery electric, opportunity charging', 
            'name': 'bus, battery electric, opportunity charging', 
            'kind': 'other',
            'exchanges': 
            [
                {
                    'amount': 0.0139, # kWh/pkm
                    'input': (banca, 'electricity mix'), 
                    'temporal distribution': [(0, 0.0139)],
                    'type': 'technosphere'
                }
            ]
        }
        , 
        (banca, 'bus, trolley bus'): 
        {
            'code': 'bus, trolley bus', 
            'name': 'bus, trolley bus', 
            'kind': 'other',
            'exchanges': 
            [
                {
                    'amount': 0.0107, # kWh/pkm
                    'input': (banca, 'electricity mix'), 
                    'temporal distribution': [(0, 0.0107)],
                    'type': 'technosphere'
                }
            ]
        }
        , 
        (banca, 'bus, hydrogen fuel cell'): 
        {
            'code': 'bus, hydrogen fuel cell', 
            'name': 'bus, hydrogen fuel cell', 
            'kind': 'other',
            'exchanges': 
            [
                {
                    'amount': 0.00106, # kg/pkm
                    'input': (banca, 'hydrogen'), 
                    'temporal distribution': [(0, 0.00106)],
                    'type': 'technosphere'
                }
            ]
        }
        , 
        (banca, 'electricity mix'): 
        {
            'code': 'electricity mix', 
            'name': 'electricity mix', 
            'kind': 'ergosphere',
            'exchanges':
            [
                {
                    'amount': sunwind, 
                    'input': (banca, 'electricity production wind, 1-3 MW turbine, onshore'), 
                    'temporal distribution': tdwind,
                    'type': 'technosphere'
                }
                ,
                {
                    'amount': sunhydr, 
                    'input': (banca, 'electricity production hydro, run-of-river'), 
                    'temporal distribution': tdhydr,
                    'type': 'technosphere'
                }
                ,
                {
                    'amount': sunphot, 
                    'input': (banca, 'electricity production, photovoltaic, 3kWp flat-roof installation, multi-Si'), 
                    'temporal distribution': tdphot,
                    'type': 'technosphere'
                }                
            ]
        }
        , 
        (banca, 'hydrogen'): 
        {
            'code': 'hydrogen', 
            'name': 'hydrogen', 
            'kind': 'ergosphere',
            'exchanges':
            [
                {
                    'amount': 52.38, # kWh/kg; η = 63% 33 kWh/kg
                    'input': (banca, 'electricity mix'), 
                    'temporal distribution': [(0, 52.38)],
                    'type': 'technosphere'
                }              
            ]
        }
        , 
        (banca, 'electricity production wind, 1-3 MW turbine, onshore'): 
        {
            'code': 'electricity production wind, 1-3 MW turbine, onshore', 
            'name': 'electricity production wind, 1-3 MW turbine, onshore', 
            'kind': 'other',
            'exchanges': 
            [
                {
                    'amount': 3.87, # MJ 
                    'input': (kwind), 
                    'temporal distribution': [(0, 3.87)],
                    'type': 'biosphere'
                }
            ]
        }
        , 
        (banca, 'electricity production hydro, run-of-river'): 
        {
            'code': 'electricity production hydro, run-of-river', 
            'name': 'electricity production hydro, run-of-river', 
            'kind': 'other',
            'exchanges': 
            [
                {
                    'amount': 3.79, 
                    'input': (khydr), 
                    'temporal distribution': [(0, 3.79)],
                    'type': 'biosphere'
                }
            ]
        }
        , 
        (banca, 'electricity production, photovoltaic, 3kWp flat-roof installation, multi-Si'): 
        {
            'code': 'electricity production, photovoltaic, 3kWp flat-roof installation, multi-Si', 
            'name': 'electricity production, photovoltaic, 3kWp flat-roof installation, multi-Si',
            'kind': 'other', 
            'exchanges': 
            [
                {
                    'amount': 3.8503, 
                    'input': (kphot), 
                    'temporal distribution': [(0, 3.8503)],
                    'type': 'biosphere'
                }
            ]
        }
    }
)


                                                                                                                                

Writing activities to SQLite3 database:
0% [############] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 06/13/2025 15:46:13
  Finished: 06/13/2025 15:46:13
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 2.12


bus fleet production

In [104]:
banca = 'busprodB'

In [105]:
bd.Database(banca).write(
    { 
        (banca, 'bus fleet production'): 
        {
            'code': 'bus fleet production', 
            'name': 'bus fleet production', 
            'kind': 'other',
            'exchanges': 
            [
                {
                    'amount': supICE, # purchaised buses/pkm
                    'input': (banca, 'bus production, internal combustion engine'), 
                    'temporal distribution': tpICE,
                    'type': 'technosphere'
                }
                , 
                {
                    'amount': supBED, 
                    'input': (banca, 'bus production, battery electric, depot charging'), 
                    'temporal distribution': tpBED,
                    'type': 'technosphere'
                }
                ,
                {
                    'amount': supBEO, 
                    'input': (banca, 'bus production, battery electric, opportunity charging'), 
                    'temporal distribution': tpBEO,
                    'type': 'technosphere'
                }
                , 
                {
                    'amount': supTB,
                    'input': (banca, 'bus production, trolley bus'), 
                    'temporal distribution': tpTB,
                    'type': 'technosphere'
                }
                , 
                {
                    'amount': supHFC,
                    'input': (banca, 'bus production, hydrogen fuel cell'), 
                    'temporal distribution': tpHFC,
                    'type': 'technosphere'
                }
            ]
        }
        , 
        (banca, 'bus production, internal combustion engine'): 
        {
            'code': 'bus production, internal combustion engine', 
            'name': 'bus production, internal combustion engine', 
            'kind': 'other',
            'exchanges':
            [
                {
                    'amount': 1.0, 
                    'input': (pbus), 
                    'temporal distribution': [(0, 1.0)],
                    'type': 'technosphere'
                }
            ]
        }
        , 
        (banca, 'bus production, battery electric, depot charging'): 
        {
            'code': 'bus production, battery electric, depot charging',
            'name': 'bus production, battery electric, depot charging',
            'kind': 'other',
            'exchanges':
            [
                {
                    'amount': 1.0, 
                    'input': (pbus), 
                    'temporal distribution': [(0, 1.0)],
                    'type': 'technosphere'
                }
                ,
                {
                    'amount': 3114.0, # kg/Bus (355 kWh)
                    'input': (batte), 
                    'temporal distribution': [(0, 3114.0)],
                    'type': 'technosphere'
                }
            ] 
        }
        , 
        (banca, 'bus production, battery electric, opportunity charging'): 
        {
            'code': 'bus production, battery electric, opportunity charging', 
            'name': 'bus production, battery electric, opportunity charging', 
            'kind': 'other',
            'exchanges':
            [
                {
                    'amount': 1.0, 
                    'input': (pbus), 
                    'temporal distribution': [(0, 1.0)],
                    'type': 'technosphere'
                }
                ,
                {
                    'amount': 1316.0, # kg/Bus (150 kWh)
                    'input': (batte), 
                    'temporal distribution': [(0, 1316.0)],
                    'type': 'technosphere'
                }
            ]
        }
        , 
        (banca, 'bus production, trolley bus'): 
        {
            'code': 'bus production, trolley bus', 
            'name': 'bus production, trolley bus', 
            'kind': 'other',
            'exchanges':
            [
                {
                    'amount': 1.0, 
                    'input': (pbus), 
                    'temporal distribution': [(0, 1.0)],
                    'type': 'technosphere'
                }
                ,
                {
                    'amount': 263.0, # kg/Bus (30 kWh)
                    'input': (batte), 
                    'temporal distribution': [(0, 263.0)],
                    'type': 'technosphere'
                }
            ]
        }
        , 
        (banca, 'bus production, hydrogen fuel cell'): 
        {
            'code': 'bus production, hydrogen fuel cell', 
            'name': 'bus production, hydrogen fuel cell', 
            'kind': 'other',
            'exchanges':
            [
                {
                    'amount': 1.0, 
                    'input': (pbus), 
                    'temporal distribution': [(0, 1.0)],
                    'type': 'technosphere'
                }
                ,
                {
                    'amount': 919.0, # kg/Bus (Traktor, GREET)
                    'input': (carbfib), 
                    'temporal distribution': [(0, 919.0)],
                    'type': 'technosphere'
                }
            ]
        }
    }
)
                                                                                                                                

Writing activities to SQLite3 database:
0% [######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 06/13/2025 15:46:15
  Finished: 06/13/2025 15:46:15
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 2.12
